# Benchmarking the code for analysis with artificial data

2025-03-26, by Kananovich:

In [1]:
import numpy as np
import scipy.stats as sts
from scipy import constants as cnst

general parameters:

In [2]:
nP = 1000 #number of particles
rho = 1510.0
dust_diam = 7.14E-6
fps = 295.0 # camera framerate in frames per second
time_step = 1.0 / fps
resol = 30000.0 #camera resolution in px/mm
resol_SI = resol * 1000.0 # camera resolution in px/meters
dust_mass = 4.0 / 3.0 * np.pi * (dust_diam / 2.0)**3 * rho #mass of the dust particles
kin_Tx = 1000.0 #kinetic temperature (in Kelvins) along the x axis
drift_x = 0
left_x_bord = 0
right_x_bord = 1752 # right border of the field of view in pixels
left_x_bord_SI = left_x_bord * 10**-6
right_x_bord_SI = right_x_bord * 10**-6 #coordinated of the right border
    #of the filed of view in meters


## Step 1. Creating an array of artificial velocities

In [3]:
arr_ind = np.arange(0,nP,1,dtype = 'int') # array of particles ID numbers

In [4]:
sigma_x = np.sqrt(cnst.k * kin_Tx / dust_mass)

In [5]:
sigma_x

0.00021903148058823087

In [6]:
art_vx_rv = sts.norm(drift_x,sigma_x)
arr_sample_vx = art_vx_rv.rvs(nP)

## Step 2. Creating an array of artificial coordinates

In [7]:
art_x_prev_rv = sts.uniform(left_x_bord_SI, right_x_bord_SI - left_x_bord_SI)
arr_sample_prev_x = art_x_prev_rv.rvs(nP)


In [8]:
arr_prev_x_inResolChunks = arr_sample_prev_x * resol_SI

In [9]:
arr_prev_x_inResolChunks_int = arr_prev_x_inResolChunks.astype(int)

In [10]:
arr_rough_prev_x = arr_prev_x_inResolChunks_int.astype(float) / resol_SI


## Step 3. Creating an array of artificial coordinates for the "next frame"

In [11]:
arr_next_x = arr_rough_prev_x + arr_sample_vx * time_step

In [12]:
arr_next_x_inResolChunks = arr_next_x * resol_SI
arr_next_x_inResolChunks_int = arr_next_x_inResolChunks.astype(int)
arr_rough_next_x = arr_next_x_inResolChunks_int.astype(float) / resol_SI

## Step 4. Calculating the restored velocities

In [13]:
arr_vx_restored = (arr_rough_next_x - arr_rough_prev_x) / time_step

## Step 5. Calculating the array of discrepancies

In [14]:
arr_discrep_x = np.abs(arr_vx_restored - arr_sample_vx)

In [15]:
arr_frac_discrep_x = np.abs(arr_discrep_x / arr_sample_vx) * 100

In [16]:
arr_frac_discrep_x.min()

0.009843056733384009

In [17]:
arr_frac_discrep_x.max()

6521.0594809052

In [18]:
arr_frac_discrep_x.mean()

18.854083308360135